In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torch.optim import Optimizer
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from pytorch_optimizer import Ranger
import numpy as np
import pandas as pd
from collections import defaultdict
from itertools import chain
from torchsummary import summary
from SE_ResNet_55 import ResNet55, BasicBlock
from SE_ResNet_68 import ResNet68, BasicBlock

In [ ]:
# Reference: https://github.com/Nikunj-Gupta/Efficient_ResNets/blob/master/lookahead.py

'''
PyTorch implement of 'Lookahead Optimizer: k steps forward, 1 step back', arXiv:1907.08610
'''

class Lookahead(Optimizer):
    def __init__(self, optimizer, k=5, alpha=0.5):
        self.optimizer = optimizer
        self.k = k
        self.alpha = alpha
        self.param_groups = self.optimizer.param_groups
        self.state = defaultdict(dict)
        self.fast_state = self.optimizer.state
        for group in self.param_groups:
            group["counter"] = 0
    
    def update(self, group):
        for fast in group["params"]:
            param_state = self.state[fast]
            if "slow_param" not in param_state:
                param_state["slow_param"] = torch.zeros_like(fast.data)
                param_state["slow_param"].copy_(fast.data)
            slow = param_state["slow_param"]
            slow += (fast.data - slow) * self.alpha
            fast.data.copy_(slow)
    
    def update_lookahead(self):
        for group in self.param_groups:
            self.update(group)

    def step(self, closure=None):
        loss = self.optimizer.step(closure)
        for group in self.param_groups:
            if group["counter"] == 0:
                self.update(group)
            group["counter"] += 1
            if group["counter"] >= self.k:
                group["counter"] = 0
        return loss

    def state_dict(self):
        fast_state_dict = self.optimizer.state_dict()
        slow_state = {
            (id(k) if isinstance(k, torch.Tensor) else k): v
            for k, v in self.state.items()
        }
        fast_state = fast_state_dict["state"]
        param_groups = fast_state_dict["param_groups"]
        return {
            "fast_state": fast_state,
            "slow_state": slow_state,
            "param_groups": param_groups,
        }

    def load_state_dict(self, state_dict):
        slow_state_dict = {
            "state": state_dict["slow_state"],
            "param_groups": state_dict["param_groups"],
        }
        fast_state_dict = {
            "state": state_dict["fast_state"],
            "param_groups": state_dict["param_groups"],
        }
        super(Lookahead, self).load_state_dict(slow_state_dict)
        self.optimizer.load_state_dict(fast_state_dict)
        self.fast_state = self.optimizer.state

    def add_param_group(self, param_group):
        param_group["counter"] = 0
        self.optimizer.add_param_group(param_group)

In [ ]:
# # Define a 55 layers ResNet with 4 residual layers
# net = ResNet55(
#     block=BasicBlock, 
#     num_blocks=[2, 2, 2, 2],               # N: number of Residual Layers | Bi:Residual blocks in Residual Layer i 
#     conv_kernel_sizes=[3, 3, 3, 3],        # Fi: Conv. kernel size in Residual Layer i 
#     shortcut_kernel_sizes=[1, 1, 1, 1] ,   # Ki: Skip connection kernel size in Residual Layer i 
#     num_channels=[64, 128, 232, 268],      # Ci: # channels in Residual Layer i 
#     avg_pool_kernel_size=8,                # P: Average pool kernel size 
#     drop=0,                                # use dropout with drop proportion 
#     squeeze_and_excitation=1               # Enable/disable Squeeze-and-Excitation Block 
#     ) 

# Define a 68 layers ResNet with 3 residual layers
net = ResNet68(
    block=BasicBlock, 
    num_blocks=[4, 4, 3],                    # N: number of Residual Layers | Bi:Residual blocks in Residual Layer i 
    conv_kernel_sizes=[3, 3, 3],             # Fi: Conv. kernel size in Residual Layer i 
    shortcut_kernel_sizes=[1, 1, 1] ,        # Ki: Skip connection kernel size in Residual Layer i 
    num_channels=64,                         # Ci: # channels in Residual Layer i 
    avg_pool_kernel_size=8,                  # P: Average pool kernel size 
    drop=0,                                  # use dropout with drop proportion 
    squeeze_and_excitation=1                 # Enable/disable Squeeze-and-Excitation Block 
    ) 

# total_params = 0 
# for x in filter(lambda p: p.requires_grad, net.parameters()):
#     total_params += np.prod(x.data.numpy().shape)
# print('Total Parameters: ', total_params) 

device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = net.to(device)

# Calculate the number of parameters for training
summary(net, input_size=(3, 32, 32))

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding = 4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize((0.4913996458053589, 0.48215845227241516, 0.44653093814849854), (0.2470322549343109, 0.24348513782024384, 0.26158788800239563))
    ])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4913996458053589, 0.48215845227241516, 0.44653093814849854), (0.2470322549343109, 0.24348513782024384, 0.26158788800239563))
    ])

train_dataset = datasets.CIFAR10(root='./data', train=True,
                                 download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False,
                                download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers = 0)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers = 0)

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0005)
# optimizer = optim.Adam(net.parameters(), lr=0.1, weight_decay=0.0005)
# optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
# optimizer = Ranger(net.parameters(), lr=0.1, weight_decay=0.0005)
optimizer = Lookahead(optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0005), k=5, alpha=0.5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 20)

In [ ]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    train_losses = [] 
    train_acc = []
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        nn.utils.clip_grad_value_(net.parameters(), clip_value=0.1) 
        optimizer.step()

        train_loss += loss.item()
        train_losses.append(train_loss)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item() 

        train_acc.append(100.*correct/total) 
        # print('Batch_idx: %d | Train Loss: %.3f | Train Acc: %.3f%% (%d/%d)'% (batch_idx, train_loss/(batch_idx+1), 100.*correct/total, correct, total)) 
    print('train_loss:', np.mean(train_losses)) 
    print('train_accuracy:', str(np.mean(train_acc)) + '%')
    train_loss_per_epoch.append(np.mean(train_losses))
    train_acc_per_epoch.append(np.mean(train_acc))
    
    
# Testing 
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    test_losses = [] 
    test_acc = [] 
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            test_losses.append(test_loss)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item() 
            test_acc.append(100.*correct/total) 
            # print('Batch_idx: %d | Test Loss: %.3f | Test Acc: %.3f%% (%d/%d)'% ( batch_idx, test_loss/(batch_idx+1), 100.*correct/total, correct, total)) 
        print('test_loss:', np.mean(test_losses)) 
        print('test_accuracy:', str(np.mean(test_acc)) + '%')
        test_loss_per_epoch.append(np.mean(test_losses))
        test_acc_per_epoch.append(np.mean(test_acc))
        
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc: 
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch
        }
        torch.save(state, './best.pth')
        best_acc = acc


In [ ]:
best_acc = 0
train_loss_per_epoch = []
test_loss_per_epoch = []
train_acc_per_epoch = []
test_acc_per_epoch = []
for epoch in range(210): 
    train(epoch)
    test(epoch)
    scheduler.step()

# Save the train and test evaluation metrics per epoch
train_test_eval = pd.DataFrame({
    "Train Loss": train_loss_per_epoch,
    "Train Accuracy": train_acc_per_epoch,
    "Test Loss": test_loss_per_epoch,
    "Test Accuracy": test_acc_per_epoch
})
train_test_eval.to_csv('./train_test_eval.csv')
train_test_eval